## Scavenger Hunt

By using the browser's *inspect* tool, I was able to find within the HTML and CSS code the first two parts of the flag:

<img src="ScavengerHunt-HTML.png" style="width: 40%;"></img>

<img src="ScavengerHunt-CSS.png" style="width: 80%;"></img>

Then, on JS script code I found a hint for the third part:

<img src="ScavengerHunt-JS.png" style="width: 50%;"></img>

Searching on web I found the following: "noindex is a rule set with a <meta> tag or HTTP response header that is used to prevent indexing of content by search engines that support the noindex rule, such as Google. When Googlebot crawls that page and gets the tag or header, Google will discard the page entirely from Google Search results, regardless of whether other sites link to it. For the noindex rule to work correctly, the page or resource must not be blocked by a **robots.txt** file". I can request the robots.txt file from the server by adding robots.txt at the end of the original URL: 

- http://mercury.picoctf.net:39698/robots.txt

So I get this answer page:

<img src="ScavengerHunt-Robots.png" style="width: 80%;"></img>

Now, this says that this is an Apache Server. On my research I discovered that there is a file called **.htaccess** that Apache uses to specify server configurations. Then, I can access this file the same way I did it with robots.txt:

- http://mercury.picoctf.net:39698/.htaccess

Obtaining:

<img src="ScavengerHunt-HTACCESS.png" style="width: 80%;"></img>

There is a well-known hidden file on MAC devices called **.DS_Store** that stores data about many things. If we search this resource on the URL we get:

> http://mercury.picoctf.net:39698/.DS_Store

<img src="ScavengerHunt-DStore.png" style="width: 80%;"></img>

> Flag: th4ts_4_l0t_0f_pl4c3s_2_lO0k_fa044227c

## Inspect HTML

With *inspector* tool I was easily able to catch the flag in the HTML document of the webpage:

<img src="InspectHTML.png" style="width:80%;"></img>

Hence: 

> FLAG: 1n5p3t0r_0f_h7ml_fd5d57bd

## Local Authority

One more time, when using *inspector* tool, I discovered that after you enter a user and password, server uses a file called `secure.js` to check if the credentials are correct. Server sent this file to me as an answer after I entered the wrong user and password. And I saw this:

<img src="LocalAuthority.png" style="width:80%;"></img>

So, when entering this credetianls in the login form I got the Flag:

> FLAG: j5_15_7r4n5p4r3n7_a8788e61

## Login

Examining the website files, I found one called `index.js`. Here the user and password are checked to verify that they match the correct values:

<img src="Login.png" style="width:80%;"></img>

The interesting thing here is that, after I entered the username and password, this function encode the values in base64 using `btoa` JS function. And then this encoded variables are compared with, obviusly other b64 encoded strings. Therefore, we can know the correct username and password by using the counterpart of btoa: `atob`, which decode b64 strings.

```js
atob("YWRtaW4") // admin
atob("cGljb0NURns1M3J2M3JfNTNydjNyXzUzcnYzcl81M3J2M3JfNTNydjNyfQ") //"picoCTF{53rv3r_53rv3r_53rv3r_53rv3r_53rv3r}"
```

## Where are the robots

**robots.txt** file is usually used so that Google does not track certain files, maybe to remain them hidden (as the hint suggests). So if I search this resource withing the original URL:

> https://jupiter.challenges.picoctf.org/problem/36474/robots.txt

I get:

<img src="WhereAreTheRobots.png" style=""></img>

Then, if I search this "hidden" resource: `477ce.html`, I get the flag:

> https://jupiter.challenges.picoctf.org/problem/36474/477ce.html

> FLAG: ca1cu1at1ng_Mach1n3s_477ce

## Cookies

The name of the problem suggests me that I have to search within the website cookies. This stuff is localized on Storage tab inside the inspector tool:

<img src="Cookies.png" style="width:80%;"></img>

Website put in the search box the placeholder "snickerdoodle". I tried that string and it worked, but there was no flag yet. Then I noticed that the value of the "name" cookie increased by one. Any other string resulted in -1. So I started to increase the value by one and refreshed the page. While increasing this value I was getting many valid cookies, but not the flag; until I reached the 18 value. Where I got the flag!

> FLAG: 3v3ry1_l0v3s_c00k135_a1f5bdb7

## Logon

Problem tell us we have to login as Joe's. But his password is imposible to obtain. Instead of that, we have to try login as other user. Hint says that any other user has free access to factory website, without entering the password. Then I just typed a random user and I logged in. Nevertheless, as a default user I don't have access to the flag. But, when I was checking the cookies I saw something interesting:

<img src="Logon.png" style="width:80%;"></img>

We can impersonate admin privilegies just by changing this cookie value to True. Then, I refreshed the page and I got the FLAG:

> FLAG: th3_c0nsp1r4cy_l1v3s_d1c24fef

## Search Source

Just by looking the files of the website with inspector tool, and digging a little bit in the HTML and CSS codes, I found the flag in `style.css` file:

<img src="SearchSource.png" style="width:70%;"></img>

> FLAG: 1nsp3ti0n_0f_w3bpag3s_ec95fa49

## It is my birthday

The problem begins by describing that there are two slightly different values that produce the same hash value when passed through the MD5 algorith. Our objective here is to find these two values, store them in two different PDF files and upload these files to the website. PDFs contents must produce the same hash value and be different at the same time.

The first hint we have is that the website is a PHP site. Considering this information, I assumed that there has to be a reference to the `md5_file` function in the php code, because this function is used to get the MD5 hash value of a file. Also I thought that there must be a comparison between the two hash values obtained from the PDF files. Hence something like this:

```php
<?=

$hash1 = md5_file($GET['file1']);
$hash2 = md5_file($GET['file2']);

if ($hash1 == $hash2){
    echo "Files have the same MD5 hash value";
}else{
    echo "Files don't have equal MD5 hash value";
}
````

At this point I was trying to find two different values that would produce a MD5 collision, i.e., the same MD5 hash value. But after some researching, I found an interesting vulnerability within PHP comparisons.

The PHP programming language exhibits a behaviour called type juggling, when performing a "loose (==)" comparison. This comparison has a set of operand conversion rules where it first tries to convert operands to a common and comparable type.

When comparing a string to a number, it will try to convert the string to a number and then perform the comparison. Even if both operands look like numbers, even if they are strings, it will convert them both to a number and perform the numeric comparison. Here are some examples of comparisons where the result will be true:

- TRUE : "0000" == int(0)
- TRUE : "0e12" == int(0)
- TRUE : "0e12" == "0e98"
- TRUE : "0xF" == "15"
- TRUE : "0e12" <= "1"

The above vulnerability has a specific implication for hashes (MD5 and SHA1) which in PHP are encoded in base 16. For example, coming in the form "0e8578698...", and since all the following characters after the "0e" are numbers, the string is interpreted as zero raised to the power of some value. Therefore, it will always have a value of zero.

Magic hashes are those that conform to the structure "0+e[0-9]+". structure, such as:
- MD5(240610708) = 0e462097431906509019562988736854
- MD5(QNKCDZO) = 0e830400451993494058024219903391

[Here](https://github.com/spaze/hashes/blob/master/md5.md) we have a list of magic hashes for MD5. We can pick these two values: 'QNKCDZO' and '240610708'. Despite the fact that they are totally different (the same with their MD5 hash values) on the following php code we see that in the if comparison they are treated as equals:

```php

<?=

$val1 = md5('240610708');
$val2 = md5('QNKCDZO');

var_dump($val1);
var_dump($val2);

if($val1 == $val2){
    echo 'Hashes are identical';
}else{
    echo 'Hashes are not identical';
}

# OUTPUT:
# string(32) "0e462097431906509019562988736854"
# string(32) "0e830400451993494058024219903391"
# Hashes are identical
```

Finally, I stored these two values in PDF files and I uploaded them to the website, obtaining this:

<img src="Birthday.png" style="width:80%;"></img>

> FLAG: c0ngr4ts_u_r_1nv1t3d_aad886b9

## Findme

Hint on this excersise says that we have to focus on redirections. If we login using the given credentials, and we check in the inspector tool network's requests, we get this info:

<img src="FindMe.png" style="width:80%;"></img>

This line: `id=bF90aGVfd2F5X2JlNzE2ZDhlfQ` indicates a redirection to another page. Curiously, if we decode this string from b64 we get: `l_the_way_be716d8e`. This is just the last part of the FLAG. During my research I realized that I have to intercept the redirection right after I send the credentials, with a tool called BurpSuite. With it I was able to intercept the redirection and I got this:

<img src="FindMe-Intercept.png" style="width:80%;"></img>

That string decoded from b64 is: `picoCTF{proxies_al`. Hence:

> FLAG: proxies_all_the_way_be716d8e